# 构建前后端分离的Web应用

## 工具清单

Cursor - <a href="https://cursor.com/">https://www.cursor.com</a>  

Docker Desktop - <a href="https://www.docker.com">https://www.docker.com</a>  

Httpie - <a href="https://www.httpie.io">https://www.httpie.io</a>  

Npm - <a href="https://www.npmjs.com">https://www.npmjs.com</a>


## 插件清单(Cursor)

Extension Pack for Java - <a href="https://open-vsx.org/extension/vscjava/vscode-java-pack">https://open-vsx.org/extension/vscjava/vscode-java-pack</a>

Spring Boot Extension Pack - <a href="https://open-vsx.org/extension/VMware/vscode-boot-dev-pack">https://open-vsx.org/extension/VMware/vscode-boot-dev-pack</a>

TypeScript (Native Preview) - <a href="https://github.com/microsoft/typescript-go">https://github.com/microsoft/typescript-go</a>

## 技术栈

### 前端

TypeScript - <a href="https://www.typescriptlang.org">https://www.typescriptlang.org</a>  

React - [https://ja.react.dev/](https://ja.react.dev/) 

Redux - [https://redux.js.org/](https://redux.js.org/)

Next.js - [https://nextjs.org/](https://nextjs.org/)

Axios - [https://axios-http.com/](https://axios-http.com/)

### 后端

Spring WebFlux - <a href="https://docs.spring.io/spring-framework/reference/web/webflux.html">https://docs.spring.io/spring-framework/reference/web/webflux.html</a>  

Spring Boot Flyway - <a href="https://docs.spring.io/spring-boot/api/rest/actuator/flyway.html">https://docs.spring.io/spring-boot/api/rest/actuator/flyway.html</a>



## 架构图

<img src="./Assets/Social Insurance Query System 2.svg" alt="Social Insurance Query System" style="max-width: 100%;">

## 搭建本地开发环境

#### 在Docker Desktop中创建PostgreSQL容器

_<span style="color: orange">docker run --name kanagawa-insurance-postgres -e POSTGRES_USER=db_user -e POSTGRES_PASSWORD=local -e POSTGRES_DB=social_insurance -d -p 5432:5432 postgres:17.4</span>_

#### 创建一个新的Spring应用

* ① 在Cursor首页按下<span style="color: orange">Shift + Command + P(MacOS)</span>或<span style="color: orange">Shift + Ctrl + P (Windows)</span>

* ② 输入 _<span style="color: orange">Spring Initializr: Create a Gradle Project</span>_

* ③ 依次设置种参数：  
   * 域：<span style="color: orange">jp.asatex.niuyuping</span>   
  
   * 包名：<span style="color: orange">social-insurance</span>  
  
   * 包类型：<span style="color: orange">Jar</span>  
  
   * Java SDK版本: <span style="color: orange">21</span>  
  
   * 依赖：<span style="color: orange">Spring Reactive Web</span>  

In [ ]:
import graphviz

# 创建一个有向图
dot = graphviz.Digraph(comment='传统 Spring Boot (Spring MVC) 流程图', 
                       graph_attr={'rankdir': 'TB', 'bgcolor': 'lightgray', 'labelloc': 't', 'label': '传统 Spring Boot (Spring MVC) 核心流程'},
                       node_attr={'shape': 'box', 'style': 'filled', 'fillcolor': 'white', 'fontname': 'sans-serif'},
                       edge_attr={'fontname': 'sans-serif'})

# 定义核心组件 (Nodes)
dot.node('A', 'Client (Browser/App)\n发送 Request', fillcolor='lightblue')
dot.node('B', 'Web 容器\n(Tomcat/Jetty)', fillcolor='lightcoral')
dot.node('C', 'Servlet 线程池\n(阻塞 I/O)', shape='cylinder', fillcolor='orange')
dot.node('D', 'DispatcherServlet\n(前端控制器)', fillcolor='lightgreen')
dot.node('E', 'Handler Mapping\n(路由匹配)', fillcolor='lightgoldenrod')
dot.node('F', 'Controller Method\n(处理请求)', fillcolor='honeydew')
dot.node('G', '业务逻辑 (Service)', fillcolor='lavender')
dot.node('H', '数据访问 (Repository)\n(阻塞 I/O)', fillcolor='pink')
dot.node('I', 'Response\n(数据模型/视图)', shape='oval', fillcolor='yellow')
dot.node('J', 'Client 接收 Response', fillcolor='lightblue')

# 定义流程 (Edges)

# 1. 请求进入 (阻塞)
dot.edge('A', 'B', label='请求到达', style='bold')
dot.edge('B', 'C', label='分配一个线程', style='dashed')
dot.edge('C', 'D', label='请求交给 DispatcherServlet')

# 2. 路由与处理
dot.edge('D', 'E', label='请求 URI')
dot.edge('E', 'F', label='匹配到 Controller 方法')

# 3. 阻塞处理链
dot.edge('F', 'G', label='调用业务 Service')
dot.edge('G', 'H', label='调用 Repository\n(线程等待 I/O 完成)', color='red', style='bold') # 强调阻塞点
dot.edge('H', 'G', label='返回数据 (阻塞)')
dot.edge('G', 'F', label='返回数据')

# 4. 响应生成与返回
dot.edge('F', 'I', label='返回 Model/View Name')
dot.edge('I', 'D', label='DispatcherServlet 处理结果')
dot.edge('D', 'C', label='返回 Response')
dot.edge('C', 'B', label='释放线程')
dot.edge('B', 'J', label='发送 Response', style='bold')

# 突出阻塞 I/O 的特性
dot.node('K', '阻塞 I/O 线程等待', shape='note', style='filled', fillcolor='red!10')
dot.edge('C', 'K', label='线程被占用', style='dashed', dir='back')

# 渲染图表
dot

In [ ]:
import graphviz

# 创建一个有向图
dot = graphviz.Digraph(comment='Spring WebFlux (响应式) 流程图', 
                       graph_attr={'rankdir': 'TB', 'bgcolor': 'lightgray', 'labelloc': 't', 'label': 'Spring WebFlux (响应式) 核心流程'},
                       node_attr={'shape': 'box', 'style': 'filled', 'fillcolor': 'white', 'fontname': 'sans-serif'},
                       edge_attr={'fontname': 'sans-serif'})

# 定义核心组件 (Nodes)
dot.node('A', 'Client (Browser/App)\n发送 Request', fillcolor='lightblue')
dot.node('B', '非阻塞 I/O (Netty/Undertow)', fillcolor='lightcoral')
dot.node('C', 'WebHandler/DispatcherHandler\n(路由匹配)', fillcolor='lightgreen')
dot.node('D', 'Controller/Router Function\n(处理程序)', fillcolor='lightgoldenrod')
dot.node('E', '业务逻辑 (Service)', fillcolor='honeydew')
dot.node('F', '数据访问 (Repository)\n返回 Mono/Flux', fillcolor='lavender')
dot.node('G', '响应式数据流\n(Flux 或 Mono)', shape='oval', fillcolor='yellow')
dot.node('H', 'Client 接收 Response', fillcolor='lightblue')

# 定义流程 (Edges)

# 1. 请求进入
dot.edge('A', 'B', label='请求到达', style='bold')
dot.edge('B', 'C', label='交给 WebFlux 栈')

# 2. 路由与处理
dot.edge('C', 'D', label='匹配到对应 Controller/Handler')

# 3. 响应式处理链 (非阻塞)
dot.edge('D', 'E', label='调用业务 Service')
dot.edge('E', 'F', label='调用 Repository (非阻塞)')
dot.edge('F', 'G', label='返回 Mono<T> 或 Flux<T>')

# 4. 数据流回传
dot.edge('G', 'E', label='数据处理')
dot.edge('E', 'D', label='数据处理')
dot.edge('D', 'C', label='返回响应流 (Publisher)')

# 5. 响应返回
dot.edge('C', 'B', label='封装响应', style='dashed')
dot.edge('B', 'H', label='非阻塞发送 Response', style='bold')

# 核心概念说明
dot.node('I', '背压 (Backpressure)', shape='box', style='dashed', fillcolor='white')
dot.edge('H', 'G', label='Client 向 Publisher 发出 Subscription', dir='back')
dot.edge('G', 'I', label='Publisher 等待 Demand 生产数据')

# 渲染图表
dot

#### 创建数据库迁移脚本

文件名：<span style="color: orange">{root}/src/main/resources/db/migration/V1__init_premium_bracket_table.sql</span>

In [ ]:
from diagrams import Diagram, Cluster, Edge, Node
from IPython.display import SVG, display # <-- 导入 SVG 和 display

# --- [ 这里是您的 C4 绘图代码，保持不变 ] ---

COLOR_USER = "#2C5B8B"
COLOR_CONTAINER_BG = "#68B0DE"
COLOR_DATABASE_BG = "#68B0DE"
COLOR_SYSTEM_BORDER = "#888888"
FONT_COLOR = "#FFFFFF"
LABEL_FONT_COLOR = "#333333"
EDGE_COLOR = "#666666"
FILENAME = "flyway_workflow_styled_final" # 定义文件名常量

with Diagram(
    "Spring Flyway 工作流程 (C4 容器图) - 定制风格",
    show=False,
    filename=FILENAME, # 使用常量
    outformat="svg",   # **设置为 SVG 格式**
    direction="LR",
    graph_attr={
        "fontsize": "14",
        "labelloc": "t",
        "fontname": "Sans-Serif",
        "ranksep": "1.5",
        "nodesep": "1",
    },
    node_attr={
        "fontname": "Sans-Serif",
        "fontsize": "12",
        "labelloc": "c",
    },
    edge_attr={
        "fontname": "Sans-Serif",
        "fontsize": "10",
        "fontcolor": LABEL_FONT_COLOR,
        "color": EDGE_COLOR,
        "penwidth": "1.5",
        "dir": "forward",
    }
):
    # 1. 外部系统 (人)
    developer = Node(
        label="开发人员/CI/CD 管道\n[Person]",
        shape="Mrecord",
        style="filled",
        fillcolor=COLOR_USER,
        fontcolor=FONT_COLOR,
        height="1.2",
        width="2.5",
        fixedsize="true",
        labelloc="c",
        fontsize="14"
    )

    # 2. 软件系统 (包含应用和数据库)
    with Cluster(
        "软件系统: My App",
        graph_attr={
            "bgcolor": "transparent",
            "pencolor": COLOR_SYSTEM_BORDER,
            "style": "dashed",
            "fontname": "Sans-Serif",
            "fontsize": "14",
            "labelloc": "b",
            "labeljust": "l",
        }
    ):
        # 2.1 应用容器
        spring_app_container = Node(
            label="Spring Boot 应用\n[Docker Container]\nFlyway 迁移器",
            shape="box",
            style="filled",
            fillcolor=COLOR_CONTAINER_BG,
            fontcolor=FONT_COLOR,
            height="2",
            width="4",
            fixedsize="true",
            labelloc="c",
            margin="0.5,0.5",
            fontsize="12"
        )
        
        # 2.2 数据存储容器
        database_container = Node(
            label="应用数据库\n[PostgreSQL]",
            shape="cylinder",
            style="filled",
            fillcolor=COLOR_DATABASE_BG,
            fontcolor=FONT_COLOR,
            height="1.5",
            width="3",
            fixedsize="true",
            labelloc="c",
            margin="0.5,0.5",
            fontsize="12"
        )
        
    # 3. 关系和工作流程
    developer >> Edge(label="1. 触发部署/启动") >> spring_app_container

    spring_app_container >> Edge(
        label="2. Flyway 检查/执行数据库迁移\n(连接数据库、对比Schema History、运行SQL、更新历史表)",
        minlen="3"
    ) >> database_container

# --- [ 关键新增部分：渲染 SVG ] ---

svg_file_path = f"{FILENAME}.svg"
try:
    # 打印消息可以移除，但为了调试保留
    # print(f"C4 容器图已成功生成到文件 {svg_file_path}") 
    
    # 使用 IPython.display.SVG 来在 Notebook 输出中渲染 SVG 文件
    display(SVG(svg_file_path))
except FileNotFoundError:
    print(f"错误：未能找到文件 {svg_file_path}。请确保 graphviz 已正确安装并运行。")

In [ ]:
#### 数据库迁移脚本内容

**文件名：** `{root}/src/main/resources/db/migration/V1__init_premium_bracket_table.sql`

```sql
-- ===========================================
-- 2025年神奈川县社会保险费等级表
-- 用于健康保险、厚生年金保险的计算
-- ===========================================

-- 创建保险费等级表
CREATE TABLE premium_bracket (
    id BIGSERIAL PRIMARY KEY,
    grade VARCHAR(20) NOT NULL UNIQUE,
    std_rem INTEGER NOT NULL,
    min_amount INTEGER NOT NULL,
    max_amount INTEGER NOT NULL,
    health_no_care NUMERIC(10, 2) NOT NULL,
    health_care NUMERIC(10, 2) NOT NULL,
    pension NUMERIC(10, 2) NOT NULL,
    created_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP,
    updated_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP
);

-- 添加表注释
COMMENT ON TABLE premium_bracket IS '2025年神奈川县社会保险费等级表，用于健康保险、厚生年金保险的计算';
COMMENT ON COLUMN premium_bracket.grade IS '等级';
COMMENT ON COLUMN premium_bracket.std_rem IS '标准报酬';
COMMENT ON COLUMN premium_bracket.min_amount IS '最小值';
COMMENT ON COLUMN premium_bracket.max_amount IS '最大值（999999999表示无上限）';
COMMENT ON COLUMN premium_bracket.health_no_care IS '健康保险费（无护理）';
COMMENT ON COLUMN premium_bracket.health_care IS '健康保险费（有护理）';
COMMENT ON COLUMN premium_bracket.pension IS '厚生年金保险费（0表示不适用）';

-- 创建索引
CREATE INDEX idx_premium_bracket_grade ON premium_bracket(grade);
CREATE INDEX idx_premium_bracket_min_max ON premium_bracket(min_amount, max_amount);

-- 插入2025年神奈川县社会保险费等级表数据
INSERT INTO premium_bracket (grade, std_rem, min_amount, max_amount, health_no_care, health_care, pension) VALUES
('1', 58000, 0, 63000, 5753.60, 6675.80, 0),
('2', 68000, 63000, 73000, 6745.60, 7826.80, 0),
('3', 78000, 73000, 83000, 7737.60, 8977.80, 0),
('4(1)', 88000, 83000, 93000, 8729.60, 10128.80, 16104.00),
('5(2)', 98000, 93000, 101000, 9721.60, 11279.80, 17934.00),
('6(3)', 104000, 101000, 107000, 10316.80, 11970.40, 19032.00),
('7(4)', 110000, 107000, 114000, 10912.00, 12661.00, 20130.00),
('8(5)', 118000, 114000, 122000, 11705.60, 13581.80, 21594.00),
('9(6)', 126000, 122000, 130000, 12499.20, 14502.60, 23058.00),
('10(7)', 134000, 130000, 138000, 13292.80, 15423.40, 24522.00),
('11(8)', 142000, 138000, 146000, 14086.40, 16344.20, 25986.00),
('12(9)', 150000, 146000, 155000, 14880.00, 17265.00, 27450.00),
('13(10)', 160000, 155000, 165000, 15872.00, 18416.00, 29280.00),
('14(11)', 170000, 165000, 175000, 16864.00, 19567.00, 31110.00),
('15(12)', 180000, 175000, 185000, 17856.00, 20718.00, 32940.00),
('16(13)', 190000, 185000, 195000, 18848.00, 21869.00, 34770.00),
('17(14)', 200000, 195000, 210000, 19840.00, 23020.00, 36600.00),
('18(15)', 220000, 210000, 230000, 21824.00, 25322.00, 40260.00),
('19(16)', 240000, 230000, 250000, 23808.00, 27624.00, 43920.00),
('20(17)', 260000, 250000, 270000, 25792.00, 29926.00, 47580.00),
('21(18)', 280000, 270000, 290000, 27776.00, 32228.00, 51240.00),
('22(19)', 300000, 290000, 310000, 29760.00, 34530.00, 54900.00),
('23(20)', 320000, 310000, 330000, 31744.00, 36832.00, 58560.00),
('24(21)', 340000, 330000, 350000, 33728.00, 39134.00, 62220.00),
('25(22)', 360000, 350000, 370000, 35712.00, 41436.00, 65880.00),
('26(23)', 380000, 370000, 395000, 37696.00, 43738.00, 69540.00),
('27(24)', 410000, 395000, 425000, 40672.00, 47191.00, 75030.00),
('28(25)', 440000, 425000, 455000, 43648.00, 50644.00, 80520.00),
('29(26)', 470000, 455000, 485000, 46624.00, 54097.00, 86010.00),
('30(27)', 500000, 485000, 515000, 49600.00, 57550.00, 91500.00),
('31(28)', 530000, 515000, 545000, 52576.00, 61003.00, 96990.00),
('32(29)', 560000, 545000, 575000, 55552.00, 64456.00, 102480.00),
('33(30)', 590000, 575000, 605000, 58528.00, 67909.00, 107970.00),
('34(31)', 620000, 605000, 635000, 61504.00, 71362.00, 113460.00),
('35(32)', 650000, 635000, 665000, 64480.00, 74815.00, 118950.00),
('36', 680000, 665000, 695000, 67456.00, 78268.00, 118950.00),
('37', 710000, 695000, 730000, 70432.00, 81721.00, 118950.00),
('38', 750000, 730000, 770000, 74400.00, 86325.00, 118950.00),
('39', 790000, 770000, 810000, 78368.00, 90929.00, 118950.00),
('40', 830000, 810000, 855000, 82336.00, 95533.00, 118950.00),
('41', 880000, 855000, 905000, 87296.00, 101288.00, 118950.00),
('42', 930000, 905000, 955000, 92256.00, 107043.00, 118950.00),
('43', 980000, 955000, 1005000, 97216.00, 112798.00, 118950.00),
('44', 1030000, 1005000, 1055000, 102176.00, 118553.00, 118950.00),
('45', 1090000, 1055000, 1115000, 108128.00, 125459.00, 118950.00),
('46', 1150000, 1115000, 1175000, 114080.00, 132365.00, 118950.00),
('47', 1210000, 1175000, 1235000, 120032.00, 139271.00, 118950.00),
('48', 1270000, 1235000, 1295000, 125984.00, 146177.00, 118950.00),
('49', 1330000, 1295000, 1355000, 131936.00, 153083.00, 118950.00),
('50', 1390000, 1355000, 999999999, 137888.00, 159989.00, 118950.00);

-- 创建触发器函数，用于自动更新 updated_at 字段
CREATE OR REPLACE FUNCTION update_updated_at_column()
RETURNS TRIGGER AS $$
BEGIN
    NEW.updated_at = CURRENT_TIMESTAMP;
    RETURN NEW;
END;
$$ language 'plpgsql';

-- 创建触发器
CREATE TRIGGER update_premium_bracket_updated_at
    BEFORE UPDATE ON premium_bracket
    FOR EACH ROW
    EXECUTE FUNCTION update_updated_at_column();
```

**注意：** 此 SQL 脚本应保存为文件 `V1__init_premium_bracket_table.sql`，放在 `src/main/resources/db/migration/` 目录下。Flyway 会在应用启动时自动执行此迁移脚本。



#### 添加配置

文件名：<span style="color: orange">{root}/src/main/resources/application.properties</span>

In [ ]:
# ===========================================
# Server Configuration
# ===========================================
# Server port configuration with environment variable override support, default port 9002
server.port=${PORT:9002}

# ===========================================
# Database Connection Configuration (R2DBC - Reactive Database Connection)
# ===========================================
# PostgreSQL database connection URL with environment variable override support
spring.r2dbc.url=${DB_URL:r2dbc:postgresql://localhost:5432/social_insurance}
# Database username with environment variable override support
spring.r2dbc.username=${DB_USER:db_user}
# Database password with environment variable override support
spring.r2dbc.password=${DB_PASSWORD:local}

# ===========================================
# Database Migration Configuration (Flyway)
# ===========================================
# Enable Flyway database migration tool
spring.flyway.enabled=true
# JDBC connection URL used by Flyway (different from R2DBC, Flyway requires JDBC)
spring.flyway.url=${FLYWAY_URL:jdbc:postgresql://localhost:5432/social_insurance}
# Flyway database username
spring.flyway.user=${DB_USER:db_user}
# Flyway database password
spring.flyway.password=${DB_PASSWORD:local}
# Database migration script location
spring.flyway.locations=classpath:db/migration
# Automatically create baseline version during migration
spring.flyway.baseline-on-migrate=true
# Baseline version number
spring.flyway.baseline-version=0
# Whether to validate scripts during migration (recommended to disable in development)
spring.flyway.validate-on-migrate=false

# ===========================================
# Logging Configuration
# ===========================================
# Enable Flyway logging
logging.level.org.flywaydb=DEBUG
logging.level.org.springframework.boot.autoconfigure.flyway=DEBUG


#### 创建端点服务

在Chat的Agent模式中输入：<span style="color: orange">请根据我的V1__init_premium_bracket_table.sql迁移脚本为我创建实体</span>  

在Chat的Agent模式中输入：<span style="color: orange">请根据我的实体为我创建Repository</span>  

在Chat的Agent模式中输入：<span style="color: orange">请根据我的Repository为我创建Service</span>

在Chat的Agent模式中输入：<span style="color: orange">请根据我的Service为我创建Controller</span>

In [ ]:
import graphviz

# 创建一个有向图
dot = graphviz.Digraph(comment='Spring Cloud 微服务 数据处理层级流程图', 
                       graph_attr={'rankdir': 'TB', 'bgcolor': 'lightgray', 'labelloc': 't', 'label': 'Spring Cloud 微服务：数据处理层级 (四层架构)'},
                       node_attr={'shape': 'box', 'style': 'filled', 'fontname': 'sans-serif'},
                       edge_attr={'fontname': 'sans-serif', 'penwidth': '1.5'})

# 定义核心层级 (Nodes)
dot.node('Controller', 'Controller\n(控制层 / API 接口)', fillcolor='#DDEEFF', shape='octagon')
dot.node('Service', 'Service\n(业务逻辑层)', fillcolor='#E0F7FA')
dot.node('Repo', 'Repository\n(持久化层 / 数据访问)', fillcolor='#FFFDE7')
dot.node('Entity', 'Entity\n(数据实体 / 领域模型)', fillcolor='#FFEBEE', shape='cylinder')
dot.node('DB', 'Database/External System\n(数据库/第三方服务)', fillcolor='#F5F5F5', shape='folder')

# 定义外部组件
dot.node('Client', 'Client Request\n(用户/API 调用)', fillcolor='#C8E6C9', shape='ellipse')
dot.node('Response', 'Response\n(JSON/XML)', fillcolor='#C8E6C9', shape='ellipse')

# --- 1. 请求流入 (自上而下) ---

# 1.1 Client -> Controller
dot.edge('Client', 'Controller', label='HTTP Request (GET/POST/PUT/DELETE)', style='bold')

# 1.2 Controller -> Service
dot.edge('Controller', 'Service', label='调用业务方法 (传入 DTO/基本参数)', color='blue')

# 1.3 Service -> Repository
dot.edge('Service', 'Repo', label='调用持久化方法 (CRUD 操作)', color='darkgreen')

# 1.4 Repository -> Entity & DB
dot.edge('Repo', 'Entity', label='映射/操作 Entity 对象')
dot.edge('Repo', 'DB', label='执行 SQL/NoSQL (底层通信)', style='dashed')


# --- 2. 响应流出 (自下而上) ---

# 2.1 DB -> Repo
dot.edge('DB', 'Repo', label='返回原始数据', dir='back', style='dashed')

# 2.2 Entity -> Repo (通常是双向操作，但这里简化为返回)
dot.edge('Entity', 'Repo', label='返回 Entity 对象', dir='back')

# 2.3 Repo -> Service
dot.edge('Repo', 'Service', label='返回 Entity 集合/对象', dir='back', color='darkgreen')

# 2.4 Service -> Controller
dot.edge('Service', 'Controller', label='返回处理结果/DTO', dir='back', color='blue')

# 2.5 Controller -> Response
dot.edge('Controller', 'Response', label='返回 HTTP Response', style='bold')

# 添加说明性的文本节点
dot.node('Info', '职责:\n1. Controller: 负责 API 暴露和参数校验。\n2. Service: 负责业务逻辑、事务控制、多 Repo 协调。\n3. Repository: 负责与数据源的简单 CRUD 交互。', 
         shape='note', fillcolor='#FFFFCC', pos='2, 4!', fontsize='10', style='rounded, filled')

# 渲染图表
dot

#### 使用Httpie测试

http POST http://localhost:9002/social-insurance/calculate monthlyAmount:=550000 age:=45